In [7]:
import random
import numpy as np
import aifc
import csv
import pandas as pd
import sys
import matplotlib.pyplot as plt

def extract_file():
    df = pd.read_csv('train.csv', header=0, sep=',')
    df.columns=['sample', 'label'] 
    return df
    
def pourc(tl, pourc):
    return pourc*tl/100

def compare_our_arrays(x,y):
    score = 0
    for i in x: 
        if x[i] == y[i]:
            score+=1
    return score

def create_label():
    df = extract_file()
    label_list = []
    for i in range(len(df)):
        label_list.append(df['label'][i])
    return label_list

def split_our_lists(): #Splits our lists 
    train = []
    validation = []
    test = []
    label_list = create_label()
    random.shuffle(label_list)
    tl = len(label_list)
    
    for i in range(tl):
        if i < pourc(tl,60):
            train.append(label_list[i])
        elif i < pourc(tl,80):
            validation.append(label_list[i])
        else:
            test.append(label_list[i])
    return train, validation, test


def show_signal(filename): #shows our signal thx to matplotlib
    liste_sample_fft = []
    s = aifc.open('Data/train/' + filename, "r")
    N = s.getnframes()
    strsig = s.readframes(N)
    y = np.fromstring(strsig, np.short).byteswap()
    FE = s.getframerate()
    A = np.fft.fft(y)
    DeltaT = 1./FE
    ampl = np.abs(A)*1./N
    freqs = np.fft.fftfreq(N, DeltaT)
    P = plt.plot(freqs[:int(N/2)], ampl[:int(N/2)]) 
    plt.ylabel("Amplitude")
    plt.xlabel("Frequence")
    plt.show()

def fct_FFT(filename) : #Fast Fourier transformation decomposes our temporal signal into frequency signal
    liste_sample_fft = []
    s = aifc.open('Data/train/' + filename, "r")
    N = s.getnframes()
    strsig = s.readframes(N)
    y = np.fromstring(strsig, np.short).byteswap()
    FE = s.getframerate()
    A = np.fft.fft(y)
    DeltaT = 1./FE
    ampl = np.abs(A)*1./N
    liste_sample_fft.append(ampl)
    return ampl

def Parcours_file_fft(listname): #Allows us to take the name of the files of train or test or validation
    df = extract_file() 
    df_train = []
    for i in range(len(listname)):
        df_train.append(fct_FFT(df["sample"][i]))
    return df_train

liste = split_our_lists() 

from sklearn.ensemble import RandomForestClassifier
Train = liste[0]
Validation = liste[1]
Train_fft = Parcours_file_fft(liste[0])
Validation_fft = Parcours_file_fft(liste[1])


In [8]:
k = 200
liste_de_test = []
for i in [2,5,7,10,15,100]:
    ClassifiedDatas = RandomForestClassifier(n_estimators = 7, max_depth = 1, min_samples_split = i)#pas un problème multidimensionnel (on prend que la fréquence)
    ClassifiedDatas.fit(Train_fft[:k], Train[:k])
    P = ClassifiedDatas.score(Validation_fft[:k], Validation[:k])
    liste_de_test.append(P)
    print('pour n_estim =',i,':',P)


pour n_estim = 2 : 0.73
pour n_estim = 5 : 0.735
pour n_estim = 7 : 0.735
pour n_estim = 10 : 0.725
pour n_estim = 15 : 0.735
pour n_estim = 100 : 0.725


In [3]:
Train = liste[0]
k = 200
Train_fft = Parcours_file_fft(liste[0])
ClassifiedDatas.fit(Train_fft[:k], Train[:k])
ClassifiedDatas.score(Train_fft[:k], Train[:k])

Y = np.array(Train[:k])
Y_predicted = ClassifiedDatas.predict(Train_fft[:k])
sum(Y == Y_predicted)


NameError: name 'ClassifiedDatas' is not defined

In [11]:
print(ClassifiedDatas.get_params(deep = True))

{'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_split': 1e-07, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10, 'n_jobs': 1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [78]:
# Effectuer avec le Validation

Validation = liste[1]
Validation_fft = Parcours_file_fft(liste[1])
print(ClassifiedDatas.score(Validation_fft[:k], Validation[:k]))

Xtst = np.array(Validation[:k])
Ytst = ClassifiedDatas.predict(Validation_fft[:k])

print(sum(Xtst == Ytst))

0.66
132


In [7]:
# ??  
#print(ClassifiedDatas.predict_proba(Train_fft[:15]))
print(ClassifiedDatas.decision_path(Train_fft[:15]))

#print(ClassifiedDatas.apply(Train_fft[:15]))


(<15x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 1720 stored elements in Compressed Sparse Row format>, array([   0,  155,  314,  455,  594,  733,  872, 1015, 1154, 1285, 1418]))
